In [ ]:
## Start
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from transformers import BartForConditionalGeneration, BartTokenizer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sentiment_model = BertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:


reviews = [
    "The lens produces sharp images, but the focus ring is a bit stiff.",
    "Color reproduction is outstanding, but the lens tends to flare in bright sunlight.",
    "The aperture control is smooth, but the lens hood is not well-designed.",
    "The bokeh effect is beautiful, but the lens is quite prone to dust."

]

ratings = [
   {'sharpness': 5, 'focus_ring': 3, 'color_reproduction': 5, 'flare': 2, 'aperture_control': 4, 'lens_hood': 2, 'bokeh': 5, 'dust_resistance': 2},
    {'sharpness': 4, 'focus_ring': 2, 'color_reproduction': 5, 'flare': 3, 'aperture_control': 5, 'lens_hood': 3, 'bokeh': 4, 'dust_resistance': 3},
    {'sharpness': 5, 'focus_ring': 4, 'color_reproduction': 4, 'flare': 2, 'aperture_control': 5, 'lens_hood': 2, 'bokeh': 5, 'dust_resistance': 2},
    {'sharpness': 4, 'focus_ring': 3, 'color_reproduction': 4, 'flare': 3, 'aperture_control': 4, 'lens_hood': 1, 'bokeh': 5, 'dust_resistance': 2}

]

In [ ]:
def sentiment_analysis(review):
    inputs = tokenizer.encode(review, return_tensors="pt", max_length=512, truncation=True)
    outputs = sentiment_model(inputs)
    probs = outputs[0].softmax(1)
    return int(np.argmax(probs)) + 1

In [ ]:
def aspect_based_sentiment_analysis(review):
    aspects = ['zoom', 'cap_quality', 'stabilization','autofocus','build_quality','weight']
    aspect_sentiments = {}

    for aspect in aspects:
        if aspect in review.lower():
            aspect_sentiments[aspect] = sentiment_analysis(review)

    return aspect_sentiments

In [ ]:
def summarize_reviews(reviews):
    inputs = bart_tokenizer(" ".join(reviews), return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
def aggregate_metrics(ratings):
    df = pd.DataFrame(ratings)
    return df.mean().to_dict()

In [ ]:
def make_decision(aspect_sentiments, aggregated_metrics):
    threshold = 3.0  # Arbitrary threshold for making decisions
    decision = "Keep"

    if any(value < threshold for value in aggregated_metrics.values()):
        decision = "Throw"
    elif any(value < threshold for value in aspect_sentiments.values()):
        decision = "Reinspect"

    return decision


In [ ]:
def sentiment_analysis(review):
    inputs = tokenizer.encode(review, return_tensors="pt", max_length=512, truncation=True)
    outputs = sentiment_model(inputs)
    probs = outputs[0].softmax(1)
    # Detach the tensor and convert it to a numpy array
    probs_np = probs.detach().numpy()
    return int(np.argmax(probs_np)) + 1  # Returns sentiment score from 1 to 5


In [ ]:
# Call the summarize_reviews function with the reviews data
summary = summarize_reviews(reviews)

summary_report = {
    "summary": summary,
    "aspect_sentiments": aggregated_aspect_sentiments,
    "aggregated_metrics": aggregated_metrics,
    "final_decision": decision
}

NameError: name 'aggregated_aspect_sentiments' is not defined

In [ ]:
aspect_sentiments = []
for review in reviews:
    # Ensure aspect_based_sentiment_analysis returns data in the expected format
    result = aspect_based_sentiment_analysis(review)
    if result:  # Check if the result is not empty
        aspect_sentiments.append(result)

# Check if aspect_sentiments is not empty before proceeding
if aspect_sentiments:
    aggregated_aspect_sentiments = pd.DataFrame(aspect_sentiments).mean().to_dict()
else:
    # Handle the case where aspect_sentiments is empty
    aggregated_aspect_sentiments = {}  # Or provide a default value

aggregated_metrics = aggregate_metrics(ratings)
summary = summarize_reviews(reviews)
decision = make_decision(aggregated_aspect_sentiments, aggregated_metrics)

summary_report = {
    "summary": summary,
    "aspect_sentiments": aggregated_aspect_sentiments,
    "aggregated_metrics": aggregated_metrics,
    "final_decision": decision
}

In [ ]:
print("Summary Report:")
for key, value in summary_report.items():
    print(f"{key.capitalize()}: {value}")

Summary Report:
Summary: The zoom function works smoothly, but the lens cap is too loose and falls off easily. Excellent image stabilization, but autofocus is slightly noisy. The build quality is solid, and the lens is a bit heavy for extended use.
Aspect_sentiments: {'zoom': 3.0, 'stabilization': 4.0, 'autofocus': 4.0}
Aggregated_metrics: {'zoom': 4.666666666666667, 'cap_quality': 2.0, 'stabilization': 4.666666666666667, 'autofocus': 2.6666666666666665, 'build_quality': 4.333333333333333, 'weight': 3.0}
Final_decision: Throw
